<a href="https://colab.research.google.com/github/orhan-akbaba/Derin-renme-Proje/blob/main/Derin_%C3%96%C4%9Frenme_Proje.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [23]:
import os
import time
import ipywidgets as widgets
from IPython.display import display, clear_output
import warnings
import subprocess


print("🛠️ Sistem hazırlanıyor...\n")
os.system("pip install -U google-generativeai selenium selenium-stealth beautifulsoup4 ipywidgets > /dev/null 2>&1")
os.system("apt-get update > /dev/null 2>&1")
os.system("apt-get install -y google-chrome-stable > /dev/null 2>&1")

try:

    chrome_ver = subprocess.check_output("google-chrome --version", shell=True).decode("utf-8").strip().split()[-1]
    os.system(f"wget -q https://storage.googleapis.com/chrome-for-testing-public/{chrome_ver}/linux64/chromedriver-linux64.zip")
    os.system("unzip -o -q chromedriver-linux64.zip")
    os.system("mv chromedriver-linux64/chromedriver /usr/bin/chromedriver")
    os.system("chmod +x /usr/bin/chromedriver")
except:
    os.system("apt-get install -y chromium-chromedriver > /dev/null 2>&1")

import google.generativeai as genai
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium_stealth import stealth
from bs4 import BeautifulSoup

warnings.filterwarnings("ignore")


def setup_driver():
    options = Options()
    options.add_argument("--headless=new")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--window-size=1920,1080")
    try:
        service = Service("/usr/bin/chromedriver")
        driver = webdriver.Chrome(service=service, options=options)
        stealth(driver, languages=["tr-TR", "tr"], vendor="Google Inc.", platform="Win32", webgl_vendor="Intel Inc.", renderer="Intel Iris OpenGL Engine", fix_hairline=True)
        return driver
    except:
        return webdriver.Chrome(options=options)

def get_data(driver, url):
    nice_data = []
    recent_data = []

    print("🌍 Siteye bağlanılıyor...")
    driver.get(url)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'html.parser')


    print("⭐ Popüler yorumlar taranıyor...")
    for i in range(1, 3):
        driver.get(f"{url}?a=nice&p={i}")
        time.sleep(1)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        divs = soup.find_all('div', {'class': 'content'}) or soup.select("ul#entry-item-list li .content")
        for d in divs:
            if len(d.text) > 20: nice_data.append(d.text)


    print("📅 Güncel yorumlar taranıyor...")
    try:
        total = 100
        pager = soup.find('div', {'class': 'pager'})
        if pager and pager.get('data-pagecount'): total = int(pager.get('data-pagecount'))
        start = max(1, total - 2)
        for i in range(start, total + 1):
            driver.get(f"{url}?p={i}")
            time.sleep(1)
            soup = BeautifulSoup(driver.page_source, 'html.parser')
            divs = soup.find_all('div', {'class': 'content'}) or soup.select("ul#entry-item-list li .content")
            for d in divs:
                if len(d.text) > 20: recent_data.append(d.text)
    except: pass

    return nice_data, recent_data


def analyze_with_model(model, entries, context):
    if not entries: return "Veri yok."
    prompt = f"""
    Sen uzman bir analistsin. Konu: Ekşi Sözlük '{context}' yorumları.
    GÖREV: Bu yorumları oku ve bir öğretmene sunulacak kalitede, AKICI BİR PARAGRAF şeklinde özetle.

    YORUMLAR:
    {" ".join(entries)[:30000]}
    """
    try:
        return model.generate_content(prompt).text
    except Exception as e:
        return f"Hata: {str(e)}"


clear_output()

header = widgets.HTML("<h2>🧠 EKŞİ SÖZLÜK AI ANALİSTİ</h2>")


api_input = widgets.Text(
    placeholder='Yeni API Anahtarını Buraya Yapıştır...',
    description='<b>API Key:</b>',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

url_input = widgets.Text(
    placeholder='Link yapıştırın...',
    description='<b>Link:</b>',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

btn = widgets.Button(description='ANALİZ ET 🚀', button_style='primary')
out = widgets.Output()

def on_click_action(b):
    with out:
        clear_output()
        key = api_input.value.strip()
        link = url_input.value.strip()


        if not key:
            print("❌ Lütfen API Anahtarını girin.")
            return
        if "eksisozluk" not in link:
            print("❌ Geçersiz Link")
            return

        print("🔧 Bağlantı kuruluyor...")


        try:
            genai.configure(api_key=key)
            model = genai.GenerativeModel('gemini-1.5-flash')
        except Exception as e:
            print(f"❌ API Anahtarı Hatası: {e}")
            return

        driver = None
        try:
            driver = setup_driver()
            nice, recent = get_data(driver, link.split("?")[0])

            if not nice and not recent:
                print("❌ Veri çekilemedi.")
            else:
                print("🤖 Yapay Zeka analiz ediyor...")

                res_nice = analyze_with_model(model, nice, "Genel Kanı")
                res_recent = analyze_with_model(model, recent, "Güncel Durum")

                print("\n" + "="*50)
                print(f"📊 RAPOR: {link.split('/')[-1]}")
                print("="*50)
                print("\n💡 GENEL KANI:\n" + "-"*30)
                print(res_nice)
                print("\n🔥 GÜNCEL DURUM:\n" + "-"*30)
                print(res_recent)

        except Exception as e:
            print(f"Sistem Hatası: {e}")
        finally:
            if driver: driver.quit()

btn.on_click(on_click_action)
display(widgets.VBox([header, api_input, url_input, btn, out]))